In [6]:
import struct

class SM4:
    # Official SM4 S-Box
    SBOX = [
        0xd6, 0x90, 0xe9, 0xfe, 0xcc, 0xe1, 0x3d, 0xb7, 0x16, 0xb6, 0x14, 0xc2, 0x28, 0xfb, 0x2c, 0x05,
        0x2b, 0x67, 0x9a, 0x76, 0x2a, 0xbe, 0x04, 0xc3, 0xaa, 0x44, 0x13, 0x26, 0x49, 0x86, 0x06, 0x99,
        0x9c, 0x42, 0x50, 0xf4, 0x91, 0xef, 0x98, 0x7a, 0x33, 0x54, 0x0b, 0x43, 0xed, 0xcf, 0xac, 0x62,
        0xe4, 0xb3, 0x1c, 0xa9, 0xc9, 0x08, 0xe8, 0x95, 0x80, 0xdf, 0x94, 0xfa, 0x75, 0x8f, 0x3f, 0xa6,
        0x47, 0x07, 0xa7, 0xfc, 0xf3, 0x73, 0x17, 0xba, 0x83, 0x59, 0x3c, 0x19, 0xe6, 0x85, 0x4f, 0xa8,
        0x68, 0x6b, 0x81, 0xb2, 0x71, 0x64, 0xda, 0x8b, 0xf8, 0xeb, 0x0f, 0x4b, 0x70, 0x56, 0x9d, 0x35,
        0x1e, 0x24, 0x0e, 0x5e, 0x63, 0x58, 0xd1, 0xa2, 0x25, 0x22, 0x7c, 0x3b, 0x01, 0x21, 0x78, 0x87,
        0xd4, 0x00, 0x46, 0x57, 0x9f, 0xd3, 0x27, 0x52, 0x4c, 0x36, 0x02, 0xe7, 0xa0, 0xc4, 0xc8, 0x9e,
        0xea, 0xbf, 0x8a, 0xd2, 0x40, 0xc7, 0x38, 0xb5, 0xa3, 0xf7, 0xf2, 0xce, 0xf9, 0x61, 0x15, 0xa1,
        0xe0, 0xae, 0x5d, 0xa4, 0x9b, 0x34, 0x1a, 0x55, 0xad, 0x93, 0x32, 0x30, 0xf5, 0x8c, 0xb1, 0xe3,
        0x1d, 0xf6, 0xe2, 0x2e, 0x82, 0x66, 0xca, 0x60, 0xc0, 0x29, 0x23, 0xab, 0x0d, 0x53, 0x4e, 0x6f,
        0xd5, 0xdb, 0x37, 0x45, 0xde, 0xfd, 0x8e, 0x2f, 0x03, 0xff, 0x6a, 0x72, 0x6d, 0x6c, 0x5b, 0x51,
        0x8d, 0x1b, 0xaf, 0x92, 0xbb, 0xdd, 0xbc, 0x7f, 0x11, 0xd9, 0x5c, 0x41, 0x1f, 0x10, 0x5a, 0xd8,
        0x0a, 0xc1, 0x31, 0x88, 0xa5, 0xcd, 0x7b, 0xbd, 0x2d, 0x74, 0xd0, 0x12, 0xb8, 0xe5, 0xb4, 0xb0,
        0x89, 0x69, 0x97, 0x4a, 0x0c, 0x96, 0x77, 0x7e, 0x65, 0xb9, 0xf1, 0x09, 0xc5, 0x6e, 0xc6, 0x84,
        0x18, 0xf0, 0x7d, 0xec, 0x3a, 0xdc, 0x4d, 0x20, 0x79, 0xee, 0x5f, 0x3e, 0xd7, 0xcb, 0x39, 0x48,
    ]

    @staticmethod
    def _rotate_left(x, n):
        return ((x << n) & 0xffffffff) | (x >> (32 - n))

    def _t_function(self, x):
        # Non-linear substitution (Tau)
        b = struct.pack(">I", x)
        sub = bytes([self.SBOX[i] for i in b])
        b_val = struct.unpack(">I", sub)[0]
        # Linear transformation (L)
        return b_val ^ self._rotate_left(b_val, 2) ^ self._rotate_left(b_val, 10) ^ \
               self._rotate_left(b_val, 18) ^ self._rotate_left(b_val, 24)

    def encrypt_block(self, block, round_keys):
        x = list(struct.unpack(">4I", block))
        for i in range(32):
            temp = x[i+1] ^ x[i+2] ^ x[i+3] ^ round_keys[i]
            x.append(x[i] ^ self._t_function(temp))
        # Reverse transformation at the end
        return struct.pack(">4I", x[35], x[34], x[33], x[32])

# Demo Usage
def run_sm4_demo():
    key = b'\x01\x23\x45\x67\x89\xab\xcd\xef\xfe\xdc\xba\x98\x76\x54\x32\x10'
    # Simplified round keys for demo (In practice, these are generated via key expansion)
    round_keys = [struct.unpack(">I", key[i%4*4:i%4*4+4])[0] for i in range(32)]
    
    msg = b"Party leads all!" # Exactly 16 bytes
    cipher = SM4()
    
    print(f"Original: {msg.decode()}")
    encrypted = cipher.encrypt_block(msg, round_keys)
    print(f"Encrypted: {encrypted}")
    print(f"Encrypted (hex): {encrypted.hex()}")
    
    # Decryption in SM4 uses the same function but round keys in reverse
    decrypted = cipher.encrypt_block(encrypted, round_keys[::-1])
    print(f"Decrypted: {msg.decode()}")

run_sm4_demo()

Original: Party leads all!
Encrypted: b'\x1b\xa0\xa0\x8c\xf6\xc5\xfbB{\xee\xd6\xb3\xbfU\xcc\xa1'
Encrypted (hex): 1ba0a08cf6c5fb427beed6b3bf55cca1
Decrypted: Party leads all!
